In [17]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_case as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)


phases = dr.CLIPS


In [18]:
models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
        "probability": [True]
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
]
# ] + train.Metrics.STATISTICAL

In [19]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


random.seed(10)

model_phases = dr.CLIPS

label_type = dr.SelfReports.AROUSAL
threshold = "fixed"

x, y = train.Train_CASE.get_case_data(
    metrics, verbose=False, label_type=label_type, threshold=threshold, 
    normalize=True, combine_phases=False, standardize=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}
ensemble_weights = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    features = {name: metrics+["lf_hf_ratio"] for name in models.keys()}
    # features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        ensemble_weights[model_name] = acc
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.70592364 0.70304683 0.71051737 0.71789923 0.70660848 0.69514786
 0.69938533 0.69861231 0.70731592 0.71310018 0.71476986 0.72014768
 0.71344611 0.71524742 0.72281007 0.72952387 0.71837623 0.70832218
 0.71382484 0.71913625 0.72480747 0.71757896 0.71148587 0.71869419
 0.72101438        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.001, 'kernel': 'sigmoid', 'probability': True}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 10, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [38  8], Predictions: [0 1], [44  2]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [38  8], Predictions: [0 1], [38  8]
Training RF ...
Model RF, Actual: [0 1], [38  8], Predictions: [0 1], [40  6]
Training XGB ...
Model XGB, Actual: [0 1], [38  8], Predictions: [0 1], [40  6]

None


[('hf_rr', 91), ('sdnn', 81), ('lf_hf_ratio', 63), ('rmssd', 60), ('lf_rr', 59), ('mean_SCL', 54), ('bpm', 48), ('SCR_rate', 25)]


[('lf_rr', 0.2332388986525138), ('hf_r

One or more of the test scores are non-finite: [0.73997311 0.74624852 0.73927228 0.73959376 0.74530212 0.74736089
 0.73523092 0.73797605 0.73804793 0.73795939 0.73632639 0.72462285
 0.73693677 0.73893466 0.74377555 0.7449371  0.7492312  0.75972643
 0.7589594  0.76065121 0.74380304 0.7413961  0.75379679 0.75774999
 0.76144469        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.001, 'kernel': 'sigmoid', 'probability': True}
LGB: {'max_depth': 4, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 4, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [34 14], Predictions: [0], [48]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [34 14], Predictions: [0 1], [39  9]
Training RF ...
Model RF, Actual: [0 1], [34 14], Predictions: [0 1], [44  4]
Training XGB ...
Model XGB, Actual: [0 1], [34 14], Predictions: [0 1], [41  7]

None


[('sdnn', 116), ('lf_hf_ratio', 106), ('rmssd', 103), ('lf_rr', 99), ('bpm', 88), ('hf_rr', 88), ('mean_SCL', 81), ('SCR_rate', 36)]


[('lf_rr', 0.20928692534472734), ('hf_rr'

...

Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.01, 'kernel': 'sigmoid', 'probability': True}
LGB: {'max_depth': 4, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [36 12], Predictions: [0 1], [40  8]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [36 12], Predictions: [0 1], [38 10]
Training RF ...
Model RF, Actual: [0 1], [36 12], Predictions: [0 1], [37 11]
Training XGB ...
Model XGB, Actual: [0 1], [36 12], Predictions: [0 1], [37 11]

None


[('hf_rr', 109), ('sdnn', 100), ('lf_hf_ratio', 94), ('bpm', 89), ('rmssd', 85), ('lf_rr', 85), ('mean_SCL', 58), ('SCR_rate', 49)]


[('hf_rr', 0.21395256315561695), ('lf_

One or more of the test scores are non-finite: [0.70557293 0.70905402 0.71525814 0.73046677 0.73316986 0.70812738
 0.72004511 0.72908414 0.72664732 0.72935452 0.68960348 0.70568186
 0.72489209 0.71924545 0.72024069 0.70702905 0.71974113 0.7232749
 0.7257886  0.71798398 0.69628246 0.71095478 0.72047963 0.72378285
 0.72655211        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 30, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [33 15], Predictions: [0 1], [40  8]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [33 15], Predictions: [0 1], [33 15]
Training RF ...
Model RF, Actual: [0 1], [33 15], Predictions: [0 1], [34 14]
Training XGB ...
Model XGB, Actual: [0 1], [33 15], Predictions: [0 1], [33 15]

None


[('hf_rr', 108), ('lf_rr', 70), ('rmssd', 65), ('sdnn', 64), ('lf_hf_ratio', 53), ('mean_SCL', 45), ('bpm', 44), ('SCR_rate', 33)]


[('lf_rr', 0.220707915447176), ('hf_rr', 0

One or more of the test scores are non-finite: [0.68805071 0.71663555 0.71670956 0.71904327 0.71722996 0.69535345
 0.71056626 0.71276504 0.71478645 0.71038999 0.71052188 0.71844955
 0.73192522 0.72830449 0.72388076 0.7176181  0.73534977 0.73797907
 0.7404145  0.733771   0.68787266 0.69907571 0.71811726 0.73213104
 0.72859059        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf', 'probability': True}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [31 16], Predictions: [0 1], [45  2]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [31 16], Predictions: [0 1], [43  4]
Training RF ...
Model RF, Actual: [0 1], [31 16], Predictions: [0 1], [44  3]
Training XGB ...
Model XGB, Actual: [0 1], [31 16], Predictions: [0 1], [45  2]

None


[('hf_rr', 77), ('sdnn', 75), ('lf_rr', 71), ('rmssd', 69), ('lf_hf_ratio', 66), ('mean_SCL', 56), ('bpm', 38), ('SCR_rate', 33)]


[('lf_rr', 0.22948917693376733), ('hf_rr', 0.

In [23]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)

from scipy.stats import mode
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


random.seed(42)

def train_predict_ensemble(ensemble_models, x_train, y_train, x_test, y_test, features, type="majority_vote", weights=None):
    y_preds = []
    if type == "majority_vote":
        # features = list(features.values())[0]
        # ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
        # ensemble = VotingClassifier(estimators=ensemble_models, voting='hard', weights=weights)
        # ensemble.fit(x_train.loc[:, features], y_train)
        # y_preds = ensemble.predict(x_test.loc[:, features])
        for model_name in ensemble_models:
            x_test_temp = x_test.loc[:, features[model_name]]
            y_pred = ensemble_models[model_name].predict(x_test_temp)
            y_preds.append(y_pred)
        y_preds = mode(y_preds, axis=0)[0]
        y_preds = np.reshape(y_preds, (y_preds.shape[1], 1))
    elif type == "weighted_avg":
        features = list(features.values())[0]
        ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
        ensemble = VotingClassifier(estimators=ensemble_models, voting='soft', weights=weights)
        ensemble.fit(x_train.loc[:, features], y_train)
        y_preds = ensemble.predict(x_test.loc[:, features])

    return y_preds


test_size = 0.15
folds = 5
ensemble_models = [
    "SVM", 
    "LGB",
    "RF",
    "XGB"
]
voting_type = "weighted_avg"

cv, x_train, y_train, x_test, y_test = train.kfold_train_test_split(x, y, test_size, is_resample=False, folds=folds, random_seed=45)
y_train = y_train.loc[:, "label"]
y_test = y_test.loc[:, "label"]

estimators = {name: best_models[name] for name in ensemble_models}
weights = [ensemble_weights[model_name] for model_name in estimators.keys()]
# weights = np.divide(weights, np.sum(weights))
# weights = np.divide(weights, len(weights))
weights = [1 for _ in range(len(list(estimators.keys())))]
y_pred = train_predict_ensemble(estimators, x_train, y_train, x_test, y_test, features, type=voting_type, weights=weights)

acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, zero_division=0)
r = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred)
print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)

	Accuracy: 0.8076923076923077
	Precision: 0.7272727272727273
	Recall: 0.4
	F1-score: 0.5161290322580645
	AUC score: 0.6741379310344827
----------------------------------------
